# Autograde a students' solution

**These instructions are currently incomplete**.

Let's say that students have turned in their notebooks with a special naming convention of `Lastname/Problem N.ipynb`, where `N` is the problem number and `Lastname` is a directory named with the student's last name. In this example, we have a few submitted assignments:

* [Bitdiddle/Problem 1.ipynb](grade_example/submitted/Bitdiddle/Problem 1.ipynb)
* [Bitdiddle/Problem 2.ipynb](grade_example/submitted/Bitdiddle/Problem 2.ipynb)
* [Hacker/Problem 1.ipynb](grade_example/submitted/Hacker/Problem 1.ipynb)
* [Hacker/Problem 2.ipynb](grade_example/submitted/Hacker/Problem 2.ipynb)

In [1]:
cd grade_example

/home/travis/build/jupyter/nbgrader/user_guide/grade_example


In [2]:
ls -R

.:
nbgrader_formgrade_config.py  submitted/

./submitted:
Bitdiddle/  Hacker/

./submitted/Bitdiddle:
Problem 1.ipynb  Problem 2.ipynb

./submitted/Hacker:
Problem 1.ipynb  Problem 2.ipynb


## Initialize the database

Before we can actually start grading, we need to actually record who the students are. We can do this using the API provided by nbgrader, which provides access to a database to store information about students and their grades:

In [3]:
# create a connection to the db using the nbgrader API
from nbgrader.api import Gradebook
gb = Gradebook("sqlite:////tmp/nbgrader_example.db")

In [4]:
# create some students and add them to the database
gb.add_student("Bitdiddle", first_name="Ben", last_name="Bitdiddle")
gb.add_student("Hacker", first_name="Alyssa", last_name="Hacker")
gb.add_student("Reasoner", first_name="Louis", last_name="Reasoner")
gb.students

[Bitdiddle, Hacker, Reasoner]

Note that the assignment should also already be in the database. An example of how to add it is given in [3 - Releasing assignments.ipynb](3 - Releasing assignments.ipynb#Recording-grade-cells).

## Run the autograder

Now that we've set up the database, we can grade the assignments. We will need to run the `nbgrader autograde` command twice (once for each student). Each time we run it, we need to specify:

* that we want to grade the notebooks in the `submitted/$student_id` directory
* that we want to copy the results to the `autograded/$student_id` directory
* what the student id is `$student_id`
* that the assignment is "Problem Set 1"
* that the database is a sqlite database at `/tmp/nbgrader_example.db`

In [5]:
%%bash

for student_id in $(ls submitted); do
    nbgrader autograde submitted/$student_id/*.ipynb \
        --build-dir=autograded/$student_id \
        --student=$student_id \
        --assignment="Problem Set 1" \
        --db="sqlite:////tmp/nbgrader_example.db"
done

[AutogradeApp] Using existing profile dir: '/home/travis/.ipython/profile_nbgrader'
[AutogradeApp] WARNING | Generating default config file: '/home/travis/.ipython/profile_nbgrader/nbgrader_autograde_config.py'
[AutogradeApp] Converting notebook submitted/Bitdiddle/Problem 1.ipynb to assignment
[AutogradeApp] Support files will be in Problem 1_files/
[AutogradeApp] Student ID: Bitdiddle
Traceback (most recent call last):
  File "/home/travis/virtualenv/python3.4.2/bin/nbgrader", line 4, in <module>
    main()
  File "/home/travis/virtualenv/python3.4.2/lib/python3.4/site-packages/nbgrader/apps/nbgraderapp.py", line 51, in main
    NbGraderApp.launch_instance()
  File "/home/travis/ipython/IPython/config/application.py", line 574, in launch_instance
    app.start()
  File "/home/travis/virtualenv/python3.4.2/lib/python3.4/site-packages/nbgrader/apps/nbgraderapp.py", line 48, in start
    super(NbGraderApp, self).start()
  File "/home/travis/ipython/IPython/config/application.py", line 2

Now if we look at the `autograded` directory, we see:

* [Bitdiddle/Problem 1.ipynb](grade_example/autograded/Bitdiddle/Problem 1.ipynb)
* [Bitdiddle/Problem 2.ipynb](grade_example/autograded/Bitdiddle/Problem 2.ipynb)
* [Hacker/Problem 1.ipynb](grade_example/autograded/Hacker/Problem 1.ipynb)
* [Hacker/Problem 2.ipynb](grade_example/autograded/Hacker/Problem 2.ipynb)

In [6]:
ls -R autograded

autograded:
Bitdiddle/  Hacker/

autograded/Bitdiddle:

autograded/Hacker:


## Replacing grade cells

If we [released the assignment with `--save-cells`](3 - Releasing assignments.ipynb#Recording-grade-cells) (i.e., we saved the grade cells into the database, then we can ensure the grade cells are the same as when the assignment was released by passing the `--overwrite-cells` flag to `nbgrader autograde`:

In [7]:
%%bash

for student_id in $(ls submitted); do
    nbgrader autograde submitted/$student_id/*.ipynb \
        --build-dir=autograded/$student_id \
        --student=$student_id \
        --assignment="Problem Set 1" \
        --db="sqlite:////tmp/nbgrader_example.db" \
        --overwrite-cells
done

[AutogradeApp] Using existing profile dir: '/home/travis/.ipython/profile_nbgrader'
[AutogradeApp] Converting notebook submitted/Bitdiddle/Problem 1.ipynb to assignment
[AutogradeApp] Support files will be in Problem 1_files/
[AutogradeApp] Student ID: Bitdiddle
[AutogradeApp] WARNING | Checksum for grade cell correct_squares has changed!
[AutogradeApp] WARNING | Checksum for grade cell correct_sum_of_squares has changed!
[AutogradeApp] WARNING | Checksum for grade cell plot_axis_limits has changed!
[AutogradeApp] WARNING | Checksum for grade cell plot_data has changed!
[AutogradeApp] Executing notebook with kernel: python3
[AutogradeApp] Writing 37064 bytes to autograded/Bitdiddle/Problem 1.ipynb
[AutogradeApp] Converting notebook submitted/Bitdiddle/Problem 2.ipynb to assignment
[AutogradeApp] Support files will be in Problem 2_files/
[AutogradeApp] Student ID: Bitdiddle
[AutogradeApp] Executing notebook with kernel: python3
[AutogradeApp] Writing 2833 bytes to autograded/Bitdiddle/P

Notice that for Ben Bitdiddle's submission for `Problem 1`, there are some cells that changed! If we look at his submission, we'll see that he actually commented out some of the test cell code, perhaps in an attempt to "trick" the grader. However, because we had saved the checksums and cell source in our database, we were able to overwrite his changes and grade his assignment with the correct tests